<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GAN" data-toc-modified-id="GAN-1">GAN</a></span><ul class="toc-item"><li><span><a href="#基本原理" data-toc-modified-id="基本原理-1.1">基本原理</a></span></li></ul></li><li><span><a href="#DCGAN" data-toc-modified-id="DCGAN-2">DCGAN</a></span><ul class="toc-item"><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-2.1">Discriminator</a></span><ul class="toc-item"><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-2.1.1">Discriminator</a></span></li></ul></li><li><span><a href="#Generator" data-toc-modified-id="Generator-2.2">Generator</a></span><ul class="toc-item"><li><span><a href="#Generator" data-toc-modified-id="Generator-2.2.1">Generator</a></span></li><li><span><a href="#损失函数" data-toc-modified-id="损失函数-2.2.2">损失函数</a></span></li></ul></li><li><span><a href="#训练及调参" data-toc-modified-id="训练及调参-2.3">训练及调参</a></span></li></ul></li></ul></div>

# GAN

## 基本原理

![GAN模型](DLND人脸生成/gan_diagram.png)

# DCGAN

## Discriminator
![DCGAN Generator](DLND人脸生成/dcgan_discriminator.png)

### Discriminator

输入：images，二维数据，28*28固定尺寸

目标：对输入的图片进行计算图片是真实图片的概率

实现方法：
    1. 选择网络层数，卷积窗尺寸kernel_size，卷积窗移动步长strides
    2. 根据输入图像尺寸和第1步选择的参数计算每一层深度和padding
    3. 根据第2步计算结果，使用tf.layers.conv2d函数设置卷积层
    4. 对每个卷积层增加bn和Leaky Relu层
    5. 将卷积层的输出reshape，使用tf.layers.dense函数增加全连接层计算logits
    6. 对logits使用sigmoid函数，将结果转换为0～1之间


## Generator
![DCGAN Generator](DLND人脸生成/dcgan_generator.png)

### Generator

输入：使用正太分布生成的随机向量，z-维向量

目标：将z-维向量转化为28 x 28 x out_channel_dim的矩阵数据

实现方法：
    1. 选择网络中卷积层数，卷积窗尺寸kernel_size
    2. 根据输入向量z的维数和目标维数计算中间每一层转置卷积的strides、padding和每层的深度
    3. 对输入的z向量连接一个全连接层
    4. reshape全连接层的输出
    5. 根据第2步计算结果使用tf.layers.conv2d_transpose设置卷积层
    6. 增加必要的bn和Leaky Relu层
    7. 对最后结果使用tanh

### 损失函数

目标： 设置训练过程需要优化的指标

实现方法：

    1. 调用generator生成一个图像
    2. 对生成图像调用discriminator计算生成图像的预测值
    3. 对真实图像调用discriminator计算真实图像的预测值
    4. 分别计算discriminator对真实图片预测结果的交叉墒和对生成图像预测结果的交叉墒，二者之和作为discriminator的最终交叉墒
    5. 计算generator生成的图像经过dscriminator分类后的交叉墒
    

## 训练及调参

**训练要点**：
    1. 选择合适batch size对数据集划分batch，逐batch进行训练
    2. 使用正太分布随机初始化input z
    3. 设置优化器，使用tf session.run开启训练任务
    4. 打印中间结果，观察训练进度

**注意要点**

- **辨别器压制生成器**

    1. 辨别器Run optimizers一次的同时生成器Run optimizers两次
    2. 使用smooth因子

- **生成器loss降不下来**
    1. 调整learning_rate防止陷入局部最优
    
- **discriminator如何设置每个卷积层的参数**
    1. 辨别器是一个CNN网络，目标是将28*28*3的图像逐步变化为14x14x64，7x7x128，4x4x256等，重点在于选择合适的strides和padding

- **generator如何设置每个卷积层的参数**
    1. 生成器的操作过程和CNN相反，使用的转置卷积进行计算，需要把向量逐步变化为a*a*A, b*b*B等形状，最后转换为28*28*out_dim的图像，重点也在于选择合适的strides和padding

**调参要点**

需要重点关注的参数包括：
1. 卷积层中的strides
2. 卷积层中的padding
3. learning_rate
4. z_dim
5. 是否使用smooth因子，如果使用，建议从小到大调整